In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import os 
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
import shap
import lime
import lime.lime_tabular

C:\Users\Enric\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Feature importance

In [3]:
folder = r'..\Dataset_simulazione'


In [4]:
target = 'techniques_mitre'

In [5]:
def prepare_dataframe(df, target, scale=False):
    # Preprocess categorical features
    X = df.drop(target, axis=1)
    y = df[target]

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    if scale:
        # Initialize the scaler
        scaler = StandardScaler()

        # Fit the scaler on the training data and transform both training and test data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        return X, X_train_scaled, y_train, X_test_scaled, y_test
        
    return X, X_train, y_train, X_test, y_test

# catBoost


In [28]:
def catboost_features_importance(file, target):
    file_path = os.path.join(folder, file)
    new_df = pd.read_csv(file_path)

    X, X_train, y_train, X_test, y_test = prepare_dataframe(new_df, target)
    
    # Fit the model
    catboost = CatBoostClassifier(verbose=0)
    catboost.fit(X_train, y_train)

 
    # Get feature importance
    importances = catboost.get_feature_importance()

    # Create a dataframe for better visualization
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
    feature_importance.sort_values(by='Importance', ascending=False, inplace=True)

    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'])
    plt.xlabel('Importance')
    plt.title('Feature Importance - CatBoost')
    plt.gca().invert_yaxis()


    # Save the figure in EPS format
    plt.savefig('catboost.pdf', format='pdf', dpi = 1200)
    plt.close()

In [29]:
catboost_features_importance("tomek_links.csv", target)